In [ ]:
# Install aiohttp package
%pip install aiohttp

# Streaming Vector Processing




## Imports

In [ ]:
# Configuration
import json
from datetime import datetime
import time
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, \
    BooleanType, IntegerType, TimestampType, LongType, \
    FloatType, ArrayType
from pyspark.sql import functions as F
import asyncio
import pandas as pd
import traceback
# sys.path.append("../../../src")

# # Import async worker from new async_util module
# from framework.utils.async_util import make_async_worker
# # Import ConfigManager from framework
# from framework.utils.config import ConfigManager



## ConfigManager

TODO: Move to framework once packaging issue is resolved

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
from typing import Dict, Any, List, Optional, Union
from pathlib import Path


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}


In [ ]:
environment = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="environment")
job_id = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="databricks_job_id")
job_run_id = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="databricks_job_run_id")
pipeline_id="acordis"

globals_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="globals_config"))
services_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="services_config"))
pipelines_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="pipelines_config"))


catalog = globals_config["catalog"]
bronze_schema_name = globals_config["bronze_schema_name"]
silver_schema_name = globals_config["silver_schema_name"]
gold_schema_name = globals_config["gold_schema_name"]


checkpoint_base_location = globals_config["checkpoint_base_location"]
checkpoint_location = f"{checkpoint_base_location}/vector_acodis_processing_checkpoints"

vector_cfg = ConfigManager.get_pipeline_steps(pipelines_config, "acordis", "pdf", "VectorProcessor") 
num_partitions = int(vector_cfg["num_partitions"])
max_files_per_trigger = int(vector_cfg["max_files_per_trigger"])
max_concurrency = int(vector_cfg["max_concurrency"])
endpoint_name = vector_cfg["endpoint_name"]
submissions_table = f"{catalog}.{bronze_schema_name}.{vector_cfg["submissions_table"]}"
parse_results_payloads_table = f"{catalog}.{silver_schema_name}.{vector_cfg["parse_results_payloads_table"]}"
output_table = f"{catalog}.{gold_schema_name}.{vector_cfg["output_table"]}"
output_submission_status = vector_cfg["submission_status"]
synthetic_mode = vector_cfg["synthetic_mode"]
micro_batch_limit = vector_cfg["micro_batch_limit"] if "micro_batch_limit" in vector_cfg else 1000
target_partitions = vector_cfg["target_partitions"] if "target_partitions" in vector_cfg else 100
service_name = vector_cfg["service"]

vector_api_config = services_config[service_name]
workspace_url = vector_api_config["workspace_url"]
token = vector_api_config["token"]
api_secret_key = vector_api_config["api_secret_key"]
api_timeout = vector_api_config["timeout_seconds"]
max_retries = vector_api_config["retry_attempts"]

openai_config = services_config["openai_service"]

print(f"Configuration loaded")
print(f"   submissions_table: {submissions_table}")
print(f"   output_table: {output_table}")
print(f"   Synthetic Data Mode: {synthetic_mode}")

# output schema for your stage
result_schema = StructType([
    StructField("submission_id", StringType(), False),
    StructField("node_id", IntegerType(), True),
    StructField("node_page", IntegerType(), True),
    StructField("node_bounding_box", StringType(), True),
    StructField("node_header", StringType(), True),
    StructField("node_parent", IntegerType(), True),
    StructField("node_content", StringType(), True),
    StructField("node_content_xml", StringType(), True),
    StructField("node_level", IntegerType(), True),
    StructField("node_class", StringType(), True),
    StructField("image_base64encoded_string", StringType(), True),
    StructField("error_traceback", StringType(), True),
    StructField("parsing_status", StringType(), True),
    StructField("processing_time_ms", LongType(), True),
    StructField("parsing_timestamp", TimestampType(), True),
    StructField("is_header", BooleanType(), True),
    StructField("databricks_parsing_job_id", StringType(), True),
    StructField("databricks_parsing_job_run_id", StringType(), True),
    StructField("parsing_batch_id", StringType(), True),
    StructField("keywords", StringType(), True),
    StructField("content_summary", StringType(), True),
    StructField("embeddings_content_summary", ArrayType(FloatType(), containsNull=False), True),
    StructField("embeddings", ArrayType(FloatType(), containsNull=False), True),
    StructField("embedding_error_traceback", StringType(), True),
    StructField("embedding_timestamp", TimestampType(), True),
    StructField("embedding_status", StringType(), True),
])

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.chains.conversation.base import LLMChain
import tiktoken

def get_llm_model(model, url, token, version):
    """returns the model to be used for summary and embedding"""

    llm_gpt = AzureChatOpenAI(
        azure_deployment=model,
        azure_endpoint=url,
        openai_api_version=version,
        temperature=0.8,
        api_key=token
    )

    return llm_gpt


def get_embedding_model(deployment, model, version, url, token, type, dimension):

    embeddings_256d = AzureOpenAIEmbeddings(
        deployment=deployment,
        model=model,
        openai_api_version=version,
        azure_endpoint=url,
        openai_api_key=token,
        openai_api_type=type,
        dimensions=dimension,
    )

    return embeddings_256d


def count_tokens(text, model="text-embedding-3-large"):
    """Count tokens for a given text using the appropriate tokenizer"""
    try:
        encoding = tiktoken.encoding_for_model(model)
        return len(encoding.encode(str(text)))
    except Exception:
        # Fallback to approximate count if tiktoken fails
        return len(str(text).split()) * 1.3  # Rough approximation
    

def content_extraction(dataset, content_extraction_type, openai_config):
    """extract content summary and key words from node/xml content"""

    model = openai_config["openai_model_chat"]
    url = openai_config["openai_url"]
    version = openai_config["openai_version"]
    token = openai_config["openai_token"]

    llm_gpt = get_llm_model(model, url, token, version)
    
    assert content_extraction_type in ["summary", "keyword"], "content needs to be summary or keyword"

    if content_extraction_type == "keyword":

        prompt_template = PromptTemplate(
            input_variables="context",
            template="""
                You are a helpful assistant and your role is to extract up to 3 comma-separated keywords from a piece of text.\n
                All extracted keywords must be mentioned in the provided text.\n
                Do not create keywords that do not appear in the provided text.\n
                Each keyword must begin with a capital letter.\n
                Provide only the comma-separated keywords and nothing else, e.g. "Keyword_1, Keyword_2, Keyword_3".\n\n
                The piece of text for which the keywords must be extracted:\n
                {context}
                """,
        )
    
    else:

        prompt_template = PromptTemplate(
            input_variables="context",
            template="""
                You are a helpful assistant and your role is to generate a summary of a text.\n
                The summary generated must contain only information mentioned in the provided text.\n
                Do not include any information that is not mentioned in the provided text.\n\n
                The text for which the summary must be generated:\n
                {context}
                """
        )


    llm_chain = LLMChain(llm=llm_gpt, prompt=prompt_template)

    total_rows = len(dataset)
    error_count = 0
    for index, row in dataset.iterrows():
        if not row["is_header"]:
            result = llm_chain.run(context=row["node_content"])
        else:
            error_count += 1
            if content_extraction_type == "keyword":
                result = "No Keywords"
            else:
                result = row["node_content"]

        if content_extraction_type == "keyword":
            dataset.at[index, "keywords"] = result
        else:
            dataset.at[index, "content_summary"] = result

        print(
            f"{content_extraction_type} extraction completed. There were {error_count} header row(s) out of {total_rows} processed."
        )
    return dataset


def create_batches(texts_with_indices, max_tokens, max_batch_size):
    """Create batches that respect both token limits and batch size limits"""
    batches = []
    current_batch = []
    current_tokens = 0

    for idx, text in texts_with_indices:
        text_tokens = count_tokens(text)

        # If single text exceeds token limit, truncate or skip
        if text_tokens > max_tokens:
            print(f"Warning: Text at index {idx} has {text_tokens} tokens, truncating...")
            # Rough truncation - could be improved with proper tokenization
            truncated_text = str(text)[:int(max_tokens * 3)]  # Rough char to token ratio
            text = truncated_text
            text_tokens = max_tokens - 100  # Leave some buffer

        # Check if adding this text exceeds limits
        if (current_tokens + text_tokens > max_tokens or
            len(current_batch) >= max_batch_size):

            if current_batch:  # Save current batch
                batches.append(current_batch)
                current_batch = []
                current_tokens = 0

        current_batch.append((idx, text, text_tokens))
        current_tokens += text_tokens

    # Add final batch
    if current_batch:
        batches.append(current_batch)

    return batches


def process_embeddings_batch(batch, embeddings_model):
    """Process a single batch and return index -> embedding mapping"""
    # Extract texts and indices
    texts = [item[1] for item in batch]
    indices = [item[0] for item in batch]

    # Single API call for entire batch
    embeddings = embeddings_model.embed_documents(texts)

    # Return mapping of index -> embedding
    return dict(zip(indices, embeddings))


def compute_embeddings(
    dataframe,
    column_name,
    new_column_name,
    openai_config
):
    """
    This function computes the embeddings of a column in a dataframe.

    Args:
        dataframe (DataFrame): The original DataFrame.
        column_name (str): Name of the column for which embeddings needs to be computed.
        new_column_name (str): Name of the new column containing the embeddings.
        embeddings_model (AzureOpenAIEmbeddings): Initialized embeddings model.

    Returns:
        DataFrame: The original DataFrame with an additional column containing the computed embeddings.

    """

    deployment = openai_config["openai_deployment_name"]
    model = openai_config["openai_model_embedding"]
    version = openai_config["openai_version"]
    url = openai_config["openai_url"]
    token = openai_config["openai_token"]
    type = openai_config["openai_api_type"]
    dimension = openai_config["openai_embedding_dimensions"]
    max_tokens = openai_config["openai_max_tokens"]
    max_batch_size = openai_config["openai_max_batch_size"]

    embeddings_model = get_embedding_model(deployment, model, version, url, token, type, dimension)

    # Prepare texts with their original indices
    texts_with_indices = [(idx, row[column_name]) for idx, row in dataframe.iterrows()]
    # Create token-aware batches
    batches = create_batches(texts_with_indices, max_tokens, max_batch_size)
    print(f"Created {len(batches)} batches for processing")

    # Process each batch and collect results
    all_embeddings = {}
    for i, batch in enumerate(batches):
        print(f"Processing batch {i+1}/{len(batches)} with {len(batch)} texts...")
        batch_results = process_embeddings_batch(batch, embeddings_model)
        all_embeddings.update(batch_results)

    # Map results back to DataFrame
    dataframe[new_column_name] = dataframe.index.map(all_embeddings)

    print(f"Completed embedding generation for {len(all_embeddings)} texts")
    return dataframe


In [ ]:
"""
Step Executor for vector processing pipeline.

"""


def execute_vector_processing(submission_id: str, use_synthetic: bool = True) -> Dict[str, Any]:
    """
    Standalone function for XML parsing - backward compatibility.

    Args:
        file_path: Path to XML file in volumes
        submission_id: Unique submission identifier
        use_synthetic: Whether to use synthetic mode

    Returns:
        Dictionary with parsing results
    """
    payload = {}
    try:
        start_time = time.time()

        if use_synthetic:
            # SYNTHETIC MODE: Fast mock processing for testing scalability
            time.sleep(0.01)  # Minimal processing time

            payload = {
                # Mock 500-dim vector
                "vector_embeddings": [0.1, 0.2, 0.3, 0.4, 0.5] * 100,
                "similarity_score": 0.85,
                "processing_model": "synthetic-vector-v1.0",
                "document_features": {
                    "complexity_score": 0.7,
                    "language": "en"
                },
                "processing_timestamp": now.isoformat(),
                "submission_id": submission_id
            }

        else:
            # PRODUCTION MODE: Real XML parsing
            #  TODO: Add code to call the vector processing API
            payload = {}

        processing_time_ms = int((time.time() - start_time) * 1000)

        return {
            'success': True,
            'parsed_data': payload,
            'processing_time_ms': processing_time_ms,
            'file_size_bytes': len(str(payload)),
            'error': None
        }

    except Exception as e:
        processing_time_ms = int(
            (time.time() - start_time) * 1000) if 'start_time' in locals() else 0
        return {
            'success': False,
            'parsed_data': None,
            'processing_time_ms': processing_time_ms,
            'file_size_bytes': 0,
            'error': str(e)
        }

## process_partition

In [ ]:
# --- PARTITION PROCESSOR ---
def process_partition(rows_iter, openai_config):

    for pdf in rows_iter:
        if pdf.empty:
            # yield an empty frame with the expected columns
            yield pd.DataFrame({f.name: pd.Series(dtype="object") for f in result_schema})
            continue

        try:
            df_rows_w_summary = content_extraction(pdf, "summary", openai_config)
            df_rows_w_keywords = content_extraction(df_rows_w_summary, "keyword", openai_config)
            df_rows_w_content_embedding = compute_embeddings(df_rows_w_keywords, "node_content", "embeddings", openai_config)
            df_rows_w_summary_embedding = compute_embeddings(df_rows_w_content_embedding, "content_summary", "embeddings_content_summary", openai_config)

            df_rows_w_summary_embedding["embedding_error_traceback"] = None
            df_rows_w_summary_embedding["embedding_status"] = "completed"
            df_rows_w_summary_embedding["embedding_timestamp"] = datetime.now()

            yield df_rows_w_summary_embedding
        except:
            pdf["keywords"] = None
            pdf["content_summary"] = None
            pdf["embeddings"] = None
            pdf["embeddings_content_summary"] = None
            pdf["embedding_error_traceback"] = traceback.format_exc()
            pdf["embedding_status"] = "content_embedding_failed"
            pdf["embedding_timestamp"] = datetime.now()

            yield pdf

## foreach_batch

In [ ]:
def foreach_batch(batch_df, batch_id):
    if batch_df.count() == 0:
      print(f"→ Batch {batch_id}: empty")
      return
    
    # Access the Spark session from the DataFrame
    spark = batch_df.sparkSession

    # Deduplicate by latest per submission_id inside the microbatch 
    w = Window.partitionBy("submission_id").orderBy(
        F.col("last_updated").desc_nulls_last())
    latest = (batch_df
              .withColumn("rn", F.row_number().over(w))
              .where("rn = 1")
              .drop("rn"))

    # Optional claim step to prevent parallel runs from double-processing:
    # Create a temp view and atomically mark a slice as 'claimed' by this job.
    latest.createOrReplaceTempView("micro_latest")

    limit_clause = f"LIMIT {int(micro_batch_limit)}" if micro_batch_limit else ""
    spark.sql(f"""
      MERGE INTO {submissions_table} t
      USING (
        SELECT submission_id
        FROM micro_latest
        WHERE submission_status IN ('xml_parsing_completed')
        {limit_clause}
      ) c
      ON t.submission_id = c.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = 'vector_indexing_in_progress',
        t.last_updated      = current_timestamp()
    """)

    # Load only the rows claimed by this run id (ensures single consumer per row)
    claimed = (spark.read.table(submissions_table)
               .where(F.col("submission_status") == "vector_indexing_in_progress")).select("submission_id").distinct()

    if claimed.count() == 0:
        print(f"→ Batch {batch_id}: No rows claimed")
        return

    df_chunks = spark.read.table("cdl_bronze_dev.poc_udp.silver_xml_payloads").join(claimed, "submission_id", "inner")

    # Repartition for wide fan-out, but keep skew keys (submission_id) as the partition column
    df_chunks = df_chunks.repartition(target_partitions, F.col("submission_id"))

    # MapPartitions on executors to call REST & write files
    # MapPartitions on executors to call REST & write files
    result_df = df_chunks.mapInPandas(
      lambda it: process_partition(it, openai_config=openai_config), schema=result_schema
    )

    print("map in pandas processing complete")
    
    result_df = result_df.withColumn("embedding_databricks_job_id", F.lit(job_id))
    result_df = result_df.withColumn("embedding_databricks_job_run_id", F.lit(job_run_id))
    result_df = result_df.withColumn("embedding_batch_id", F.lit(batch_id).cast("string")) 
    result_df.write.format("delta").mode("append").saveAsTable(output_table)

    # update the status of the submission
    result_df.select("submission_id", "embedding_status").distinct().createOrReplaceTempView("submission_updates")
    # MERGE results back, clear claim, set result metadata
    spark.sql(f"""
      MERGE INTO {submissions_table} AS t
      USING submission_updates u
      ON t.submission_id = u.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = u.embedding_status,
        t.last_updated = current_timestamp()
    """)


## Read Source Data
Read data from submissions table for submission_status=submitted

In [ ]:
# CDF streaming source - using submissions table
submissions_stream = (
    spark.readStream
         .format("delta")
         .option("readChangeFeed", "true")
         .table(submissions_table)
         .where(F.col("_change_type").isin("insert", "update_postimage"))
         .where(F.col("submission_status") == "xml_parsing_completed")
         .where(F.col("pipeline_id") == "acodis")
         .selectExpr("* EXCEPT(_change_type, _commit_version, _commit_timestamp)")
)

print("CDF streaming source configured for submissions table")


## Write Data 
1. Write data to silver table
2. On success, update submission_status=xml_extraction_completed

In [ ]:
print("Starting vector processing streaming job...")
print(f"   Mode: {'Synthetic Data' if synthetic_mode else 'Real API'}")

streaming_query = (submissions_stream.writeStream
                   .foreachBatch(foreach_batch)
                   .option("checkpointLocation", checkpoint_location)
                   .option("maxFilesPerTrigger", max_files_per_trigger)
                   .trigger(availableNow=True)
                   .outputMode("append")
                   .start())

print("Vector processing streaming job started")

try:
    streaming_query.awaitTermination()
    print("Vector Processing Job Completed completed successfully")    
    
except Exception as e:
    print(f"Streaming job error: {str(e)}")
    streaming_query.stop()
    raise
